directory --> PS C:\Users\31615\Desktop\thesis\SPARQL_LLM>
venv --> thesis

ALWAYS!!!!
#activate by .\thesis\Scripts\activate
#deactivate after u done working 

In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # This should return False since you're using CPU

2.4.0+cpu
False


Since LLaMA is a causal language model, you should use the AutoModelForCausalLM class instead of AutoModelForSeq2SeqLM.

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("openlm-research/open_llama_7b_v2", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("openlm-research/open_llama_7b_v2")

c:\Users\31615\Desktop\thesis\SPARQL_LLM\thesis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https:/

In [3]:
from datasets import load_dataset

# Load dataset from a JSON file
data = load_dataset('json', data_files='data/qald9/train.json')

In [5]:
import json
import os

def load_json_files(json_files):
    documents = []
    for file in json_files:
        with open(file, 'r') as f:
            data = json.load(f)
            for item in data:
                # Assuming 'text' field contains the content to be indexed
                documents.append(item['text'])
    return documents

# Define paths to your datasets
dataset_dirs = [
    'data/lcquad2',
    'data/qald9',
    'data/vquanda'
]

all_documents = []

# Loop through each dataset and load all the documents
for dataset_dir in dataset_dirs:
    json_files = [
        os.path.join(dataset_dir, 'dep_mapping.json'),
        os.path.join(dataset_dir, 'pos_mapping.json'),
        os.path.join(dataset_dir, 'test.json'),
        os.path.join(dataset_dir, 'train.json'),
        os.path.join(dataset_dir, 'val.json')
    ]
    
    # Load documents from current dataset and add them to the master list
    documents = load_json_files(json_files)
    all_documents.extend(documents)  # Combine all documents from different datasets

# Now, all_documents contains the combined data from all three datasets


TypeError: string indices must be integers

In [6]:
import json
import os

def load_json_files(json_files):
    documents = []
    for file in json_files:
        if not os.path.exists(file):
            print(f"Warning: {file} not found. Skipping.")
            continue
        
        with open(file, 'r') as f:
            try:
                data = json.load(f)
                for item in data:
                    # Ensure the item has the 'text' field
                    if 'text' in item:
                        documents.append(item['text'])
                    else:
                        print(f"Warning: 'text' field not found in {file}. Skipping this item.")
            except json.JSONDecodeError:
                print(f"Error: Could not decode JSON from {file}. Skipping.")
    return documents

# Define paths to your datasets
dataset_dirs = [
    'data/lcquad2',
    'data/qald9',
    'data/vquanda'
]

all_documents = []

# Loop through each dataset and load all the documents
for dataset_dir in dataset_dirs:
    json_files = [
        os.path.join(dataset_dir, 'dep_mapping.json'),
        os.path.join(dataset_dir, 'pos_mapping.json'),
        os.path.join(dataset_dir, 'test.json'),
        os.path.join(dataset_dir, 'train.json'),
        os.path.join(dataset_dir, 'val.json')
    ]
    
    # Load documents from current dataset and add them to the master list
    documents = load_json_files(json_files)
    all_documents.extend(documents)  # Combine all documents from different datasets

# Now, all_documents contains the combined data from all three datasets
print(f"Total documents loaded: {len(all_documents)}")


Total documents loaded: 0


In [ ]:
from transformers import RagRetriever, RagTokenizer, RagSequenceForGeneration

# Load a pre-trained RAG model
tokenizer = RagTokenizer.from_pretrained("facebook/rag-sequence-nq")
model = RagSequenceForGeneration.from_pretrained("facebook/rag-sequence-nq")

# Initialize the retriever with the combined documents
retriever = RagRetriever.from_pretrained(
    "facebook/rag-sequence-nq",
    index_name="custom",
    passages=all_documents  # Use combined documents
)

# Save the retriever for later use
retriever.save_pretrained("path/to/save/retriever")


In [4]:
import torch
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration

# Load the RAG tokenizer and retriever
rag_tokenizer = RagTokenizer.from_pretrained("facebook/rag-sequence-nq")
rag_retriever = RagRetriever.from_pretrained("facebook/rag-sequence-nq", use_dummy_dataset=True)

# Initialize the RAG model
rag_model = RagSequenceForGeneration.from_pretrained("facebook/rag-sequence-nq")

# Tokenize the questions and contexts
inputs = rag_tokenizer(data['question'], return_tensors="pt", padding=True, truncation=True)
contexts = rag_tokenizer(data['context'], return_tensors="pt", padding=True, truncation=True)

# Retrieve the top-k documents using FAISS
retrieved_docs = rag_retriever(inputs['input_ids'], return_tensors="pt")

# Generate the answers using the retrieved documents
generated_ids = rag_model.generate(input_ids=inputs['input_ids'], context_input_ids=retrieved_docs['context_input_ids'])
generated_texts = rag_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

for question, generated_text in zip(data['question'], generated_texts):
    print(f"Question: {question}")
    print(f"Generated Answer: {generated_text}\n")


c:\Users\31615\Desktop\thesis\SPARQL_LLM\thesis\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\31615\.cache\huggingface\hub\models--facebook--rag-sequence-nq. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
The tokenizer class you load from this checkpoint is not the same type as the class this fun

ImportError: 
RagRetriever requires the 🤗 Datasets library but it was not found in your environment. You can install it with:
```
pip install datasets
```
In a notebook or a colab, you can install it by executing a cell with
```
!pip install datasets
```
then restarting your kernel.

Note that if you have a local folder named `datasets` or a local python file named `datasets.py` in your current
working directory, python may try to import this instead of the 🤗 Datasets library. You should rename this folder or
that python file if that's the case. Please note that you may need to restart your runtime after installation.


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
)
# Define the trainer
trainer = Seq2SeqTrainer(
    model=rag_model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
)
# Fine-tune the model
trainer.train()